# Data cleaning

### Importing packages

In [ ]:
import os
import shutil as sh
from pathlib import Path

### Preliminary operations

In [ ]:
DATA_FOLDER = "clean_data"
OUTPUT_FOLDER = "output"
STATIONS_FILE = "wa_alt_fuel_stations.csv"
CARS_FILE = "Electric_Vehicle_Population_Data.csv"
WAGE_FILE = "20zpallnoagi.csv"

# Get the absolute path
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())
try:
    # Remove the existing folder
    sh.rmtree(DATA_FOLDER)
    sh.rmtree(OUTPUT_FOLDER)
except FileNotFoundError:
    print("--- No folder to remove ---")

# Create new folder for clean data
os.mkdir(DATA_FOLDER)
print(f"Folder '{DATA_FOLDER}' created.")
os.mkdir(OUTPUT_FOLDER)
print(f"Folder '{OUTPUT_FOLDER}' created.")

DATA_FOLDER += "\\"
OUTPUT_FOLDER += "\\"

#### Correcting and filtering fuel stations file

In [ ]:
file = open("stations_pub+priv_open (Nov 27 2023).csv", "r", encoding="utf-8")                  # Input file
wa_fuel_stations = wa_e_stations = open(DATA_FOLDER + STATIONS_FILE, "w", encoding="utf-8")     # Output file

# Write CSV headers
wa_fuel_stations.write(file.readline())

row = file.readline()               # Read first line
while (row2 := file.readline()):
    row2_error = False              

    # If the row is interrupted, recover it (there can be multiple interruption)
    while("ELEC" not in row2):
        row2_error = True
        index = row2.find('",')                                     # Find the end of last interrupted string, if exists
        row = row.strip() + row2[index if index != -1 else 0 : ]    # Concatenate the row begin with the second part
        row2 = file.readline()

    if ",WA," in row: wa_e_stations.write(row)
    row = row2                                                      # Check on next cycle

file.close()
wa_e_stations.close()

# RDF creation

### Importing packages

In [ ]:
import pandas as pd
from urllib.parse import quote
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import XSD

## ZIP codes, cities, counties

In [ ]:
places = pd.read_csv("wa_zips_cities_counties.csv", sep=",")

ECO = Namespace("http://www.dei.unipd.it/~poor6/db2/ontologies/2023/electricCars#")

graph = Graph()
graph.bind("elec", ECO)

In [ ]:
%%time

for index, row in places.iterrows():
    ZIP = URIRef(ECO[str(row['Zipcode'])])
    City = URIRef(ECO[quote(str(row['City']))])
    County = URIRef(ECO[quote(str(row['County']))])

    graph.add((ZIP, RDF.type, ECO.ZIP))
    graph.add((City, RDF.type, ECO.City))
    graph.add((County, RDF.type, ECO.County))

    graph.add((ZIP, ECO["ofCity"], City))
    graph.add((City, ECO["belongsTo"], County))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(OUTPUT_FOLDER + 'locations.ttl', 'w') as file:
    file.write(graph.serialize(format='turtle'))

## Washington electrical stations

In [ ]:
stations = pd.read_csv(DATA_FOLDER + STATIONS_FILE, sep=",")

graph = Graph()
graph.bind("elec", ECO)

In [ ]:
%%time

graph.add((ECO.PublicStation, RDF.type, ECO.Station))
graph.add((ECO.PrivateStation, RDF.type, ECO.Station))

for index, row in stations.iterrows():
    Station = URIRef(ECO[str(index)])     # Create node (prefix + id)

    if "Private" in row['Groups With Access Code']:
        graph.add((Station, RDF.type, ECO.PrivateStation))
    elif "Public" in row['Groups With Access Code']:
        graph.add((Station, RDF.type, ECO.PublicStation))
    else:
        graph.add((Station, RDF.type, ECO.Station))

    # Triples
    graph.add((Station, ECO['hasName'], Literal(row['Station Name'], datatype=XSD.string)))
    
    if " " in row['ZIP']: 
        print(f"Error in ZIP '{row['ZIP']}', skipped")
        continue

    ZipCode = URIRef(ECO[row['ZIP']])
    graph.add((Station, ECO['locatedIn'], ZipCode))


In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(OUTPUT_FOLDER + 'stations.ttl', 'w') as file:
    file.write(graph.serialize(format='turtle'))

## Cars

In [ ]:
cars = pd.read_csv(CARS_FILE, sep=",")
cars.info()

graph = Graph()      #Graph redefine, so to keep separate working spaces.
graph.bind("elec", ECO)

In [ ]:
%%time


#urllib.parse.quote(...)
#urllib.parse.unquote(...)

for index, row in cars.iterrows():
    Car = URIRef(ECO[str(row['DOL Vehicle ID'])])     # Create node (prefix + id)
    
    EncodedModel = quote(row['Model'])
    ModelYear = URIRef(ECO[ EncodedModel + str(row['Model Year']) ])  #2012 'Panda' => Panda2012 <---USO ENC.MOD.
    
    # Car-instance specific insertions
    graph.add((Car, RDF.type, ECO.Car))
    graph.add((Car, ECO['hasRange'], Literal(row['Electric Range'], datatype=XSD.integer) ))
    graph.add((Car, ECO['hasModelYear'], ModelYear))
    
    graph.add((ModelYear, RDF.type, ECO.ModelYear))
    graph.add((ModelYear, ECO['year'], Literal(row['Model Year'], datatype=XSD.gYear) ))
    graph.add((ModelYear, ECO['hasMSRP'], Literal(row['Base MSRP'], datatype=XSD.integer) ))
    #Model = URIRef(ECO[row['Model']])
    Model = URIRef(ECO[EncodedModel])  #<--- USO ENC.MOD.
    graph.add((ModelYear, ECO['ofModel'], Model))
    graph.add((Model, RDF.type, ECO.Model))
    #Maker = URIRef(ECO[row['Make']])
    Maker = URIRef(ECO[quote(row['Make'])])   #<---ENCODED
    graph.add((Model, ECO['madeBy'], Maker))
    graph.add((Maker, RDF.type, ECO.Maker))
        
    

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(OUTPUT_FOLDER + 'cars.ttl', 'w') as file:
    file.write(graph.serialize(format='turtle'))


## Wage per ZIP code

In [ ]:
salaries = pd.read_csv(WAGE_FILE, sep=",")
salaries.info()

graph = Graph()      #Graph redefine, so to keep separate working spaces.
graph.bind("elec", ECO)

In [ ]:
%%time

for index, row in salaries.iterrows():
    # Exclude non Washington data
    if "WA" not in row['STATE'] or '98' not in str(row['ZIPCODE']):
        continue

    Zipcode = URIRef(ECO[str(row['ZIPCODE'])])
    
    agi = float(row['A00100'])/float(row['N2'])

    graph.add((Zipcode, ECO['hasAgi'], Literal(agi, datatype=XSD.float)))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(OUTPUT_FOLDER + 'agi.ttl', 'w') as file:
    file.write(graph.serialize(format='turtle'))